In [22]:
%pip install torch
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
import torch
import torchvision.models as models

CHECKPOINT_URL = 'https://huggingface.co/alexsu52/mobilenet_v2_imagenette/resolve/main/pytorch_model.bin'
IMAGENETTE_CLASSES = 10

model = models.mobilenet_v2(num_classes=IMAGENETTE_CLASSES)  
        
checkpoint = torch.hub.load_state_dict_from_url(CHECKPOINT_URL, progress=False)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [24]:
# Input to the model
x = torch.randn(1, 3, 224, 224, requires_grad=True)
torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "imagenette.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [25]:
test = model(x)
print(test.shape)
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

torch.Size([1, 10])
2236682


# snarking the exported imagenette

In [26]:
%pip install ezkl


Note: you may need to restart the kernel to use updated packages.


In [28]:
import ezkl
import numpy as np

In [29]:
model_path = os.path.join('/Users/lnm/montreal/zk-hack/ezkl-example/models/ezkl-lenet.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs




In [30]:
#### generate dummy input data
import numpy as np
import json
dummy_input = np.random.rand(1, 1, 32, 32).astype(np.float32)

data = {'input_data': [dummy_input.reshape([-1]).tolist()]}
with open('input.json', 'w') as f:
    json.dump(data, f)

####

In [31]:

ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

True

In [32]:
ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
ezkl.get_srs("settings.json")
ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

True

# prove and verify

In [34]:
await ezkl.gen_witness(data_path, compiled_model_path, witness_path)


{'inputs': [['...',
   ...]],
 'outputs': [['0800000000000000000000000000000000000000000000000000000000000000',
   '0500000000000000000000000000000000000000000000000000000000000000',
   '0b00000000000000000000000000000000000000000000000000000000000000',
   'faffffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430',
   '0500000000000000000000000000000000000000000000000000000000000000',
   '1200000000000000000000000000000000000000000000000000000000000000',
   '0500000000000000000000000000000000000000000000000000000000000000',
   '0c00000000000000000000000000000000000000000000000000000000000000',
   '0500000000000000000000000000000000000000000000000000000000000000',
   '0100000000000000000000000000000000000000000000000000000000000000']],
 'max_lookup_inputs': 0,
 'min_lookup_inputs': 0,
 'max_range_size': 128}

In [35]:

proof_path = os.path.join('proof.json')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )